In [10]:
##COLLECTING DATA FOR PULL REQUEST EVENTS
import psycopg2
import pandas as pd 
import sqlalchemy as salc
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import datetime
from collections import defaultdict
import json
import ast
from sqlalchemy import inspect
warnings.filterwarnings('ignore')

inpt = { "connection_string": "sqlite:///:memory:", "database": "augur_osshealth", "host": "augur.osshealth.io", "password": "covfefe2020", "port": 5432, "schema": "augur_data", "user": "chaoss", "user_type": "read_only"}

config = ast.literal_eval(json.dumps(inpt))


database_connection_string = 'postgresql+psycopg2://{}:{}@{}:{}/{}'.format(config['user'], config['password'], config['host'], config['port'], config['database'])

dbschema='augur_data'
engine = salc.create_engine(
    database_connection_string,
    connect_args={'options': '-csearch_path={}'.format(dbschema)})

##IF YOU WANT TO CHANGE REPO NUMBER OR ADD MORE
repo_set = [25440]
pr_all = pd.DataFrame()
for repo_id in repo_set:
    
    pr_query = salc.sql.text(f"""
                    SELECT
                        repo.repo_id as repo_id,
                        repo.repo_name as repo_name,
                        MAX(pull_requests.pull_request_id) as pull_request_id,
                        pull_request_events.cntrb_id as pr_cntrb_id
                    FROM  
                        repo,
                        pull_requests,
                        pull_request_events
                    WHERE
                        repo.repo_id = {repo_id}
                        AND
                        pull_requests.repo_id = repo.repo_id
                        AND
                        pull_requests.pull_request_id = pull_request_events.pull_request_id
                    GROUP BY
                        repo.repo_id,
                        repo.repo_name,
                        pull_requests.pull_request_id,
                        pull_request_events.cntrb_id
                    ORDER BY
                        pull_request_id DESC,
                        repo_id           
            """)
    
pr_all = pd.read_sql(pr_query, con=engine)
display(pr_all)
pr_all.dtypes

,repo_id,repo_name,pull_request_id,pr_cntrb_id
0,25440,augur,214080,280547
1,25440,augur,214079,280547
2,25440,augur,214078,279417
3,25440,augur,214078,280551
4,25440,augur,214077,280547
...,...,...,...,...
1285,25440,augur,210010,277693
1286,25440,augur,210009,277693
1287,25440,augur,210008,277693
1288,25440,augur,210007,277693


repo_id             int64
repo_name          object
pull_request_id     int64
pr_cntrb_id         int64
dtype: object

In [2]:
#COLLECTING PULL COMMIT DATA
repo_set = [25440]
pc_all = pd.DataFrame()
for repo_id in repo_set:
    
    pc_query = salc.sql.text(f"""
                    SELECT
                        repo.repo_id as repo_id,
                        repo.repo_name as repo_name,
                        MAX(pull_request_commits.pull_request_id) as pull_request_commits_id,
                        pull_request_events.cntrb_id as pc_cntrb_id,
                        pull_requests.pull_request_id as pull_request_id
                    FROM  
                        repo,
                        pull_request_commits,
                        pull_request_events,
                        pull_requests
                    WHERE
                        repo.repo_id = {repo_id}
                        AND
                        pull_request_commits.pull_request_id = pull_requests.pull_request_id
                        AND
                        pull_request_commits.pull_request_id = pull_request_events.pull_request_id
                    GROUP BY
                        repo.repo_id,
                        repo.repo_name,
                        pull_request_commits.pull_request_id,
                        pull_request_events.cntrb_id,
                        pull_requests.pull_request_id
                    ORDER BY
                        pull_request_commits_id DESC,
                        pull_request_id,
                        repo_id           
            """)
    
pc_all = pd.read_sql(pc_query, con=engine)
display(pc_all)
pc_all.dtypes

,repo_id,repo_name,pull_request_commits_id,pc_cntrb_id,pull_request_id
0,25440,augur,219264,280707,219264
1,25440,augur,219037,281105,219037
2,25440,augur,219032,279366,219032
3,25440,augur,219027,279422,219027
4,25440,augur,218728,281105,218728
...,...,...,...,...,...
16005,25440,augur,209025,280515,209025
16006,25440,augur,209024,278680,209024
16007,25440,augur,209023,278680,209023
16008,25440,augur,209022,278680,209022


repo_id                     int64
repo_name                  object
pull_request_commits_id     int64
pc_cntrb_id                 int64
pull_request_id             int64
dtype: object

In [3]:
#COLLECTING ISSUES EVENT DATA
repo_set = [25440]
is_all = pd.DataFrame()
for repo_id in repo_set:
    
    is_query = salc.sql.text(f"""
                    SELECT
                        repo.repo_id as repo_id,
                        repo.repo_name as repo_name,
                        MAX(issue_events.issue_id) as issue_event_id,
                        issue_events.cntrb_id as issue_cntrb_id,
                        issues.issue_id as issue_id
                    FROM  
                        repo,
                        issues,
                        issue_events
                    WHERE
                        repo.repo_id = {repo_id}
                        AND
                        issues.repo_id = repo.repo_id
                        AND
                        issue_events.issue_id = issues.issue_id
                    GROUP BY
                        repo.repo_id,
                        repo.repo_name,
                        issue_events.issue_id,
                        issues.issue_id,
                        issue_events.cntrb_id
                    ORDER BY
                        issue_event_id DESC,
                        repo_id           
            """)
    
is_all = pd.read_sql(is_query, con=engine)
display(is_all)
is_all.dtypes

,repo_id,repo_name,issue_event_id,issue_cntrb_id,issue_id
0,25440,augur,343702,280547,343702
1,25440,augur,343701,280547,343701
2,25440,augur,343700,280547,343700
3,25440,augur,343699,280547,343699
4,25440,augur,343698,280547,343698
...,...,...,...,...,...
2380,25440,augur,339456,280498,339456
2381,25440,augur,339456,279781,339456
2382,25440,augur,339455,280551,339455
2383,25440,augur,339455,280498,339455


repo_id            int64
repo_name         object
issue_event_id     int64
issue_cntrb_id     int64
issue_id           int64
dtype: object

In [9]:
#COLLECTING DATA FROM EACH PANDA DF INTO 3 DICTIONARIES
contributor_quality_dict = {}
pr_dict = {}

for i in range(len(pr_all.pull_request_id)):
    if pr_all.pr_cntrb_id[i] in pr_dict.keys():
        pr_dict[pr_all.pr_cntrb_id[i]] += 1
    else:
        pr_dict[pr_all.pr_cntrb_id[i]] = 1
pr_dict_items = pr_dict.items()
print('Pull Request Events Collected for Each Contributor ID')
for item in pr_dict_items:
    print(item)


pc_dict = {}

for i in range(len(pc_all.pull_request_commits_id)):
    if pc_all.pc_cntrb_id[i] in pc_dict.keys():
        pc_dict[pc_all.pc_cntrb_id[i]] += 1
    else:
        pc_dict[pc_all.pc_cntrb_id[i]] = 1
print('\n\n\n\n\n\n')
pc_dict_items = pc_dict.items()
print('Commit Events Collected for Each Contributor ID')
for item in pc_dict_items:
    print(item)

is_dict = {}

for i in range(len(is_all.issue_event_id)):
    if is_all.issue_cntrb_id[i] in is_dict.keys():
        is_dict[is_all.issue_cntrb_id[i]] += 1
    else:
        is_dict[is_all.issue_cntrb_id[i]] = 1
print('\n\n\n\n\n\n')
is_dict_items = is_dict.items()
print('Issue Events Collected for Each Contributor ID')
for item in is_dict_items:
    print(item)





Pull Request Events Collected for Each Contributor ID
(280547, 67)
(279417, 5)
(280551, 37)
(280552, 15)
(280640, 1)
(280642, 1)
(280645, 1)
(280641, 1)
(280631, 1)
(280588, 1)
(280578, 3)
(279422, 5)
(280604, 1)
(280550, 6)
(280603, 1)
(280599, 2)
(280579, 3)
(279418, 4)
(280553, 2)
(280474, 1)
(280521, 27)
(279407, 4)
(280522, 5)
(280523, 329)
(280495, 9)
(280498, 260)
(280507, 221)
(279145, 4)
(279203, 1)
(277847, 4)
(278656, 6)
(279202, 1)
(280517, 1)
(279200, 1)
(279199, 1)
(277917, 56)
(279192, 3)
(279781, 13)
(279251, 3)
(277911, 2)
(277705, 11)
(277906, 3)
(280508, 6)
(277915, 3)
(277702, 2)
(277768, 7)
(277914, 1)
(277123, 1)
(277742, 2)
(277751, 3)
(277749, 7)
(277767, 2)
(277760, 2)
(277803, 22)
(277896, 1)
(277884, 1)
(277883, 1)
(277877, 3)
(277870, 1)
(277863, 1)
(277858, 1)
(277856, 1)
(277850, 1)
(277687, 19)
(278680, 1)
(277840, 1)
(280513, 1)
(277694, 15)
(277699, 1)
(280515, 2)
(277831, 1)
(277698, 2)
(277830, 1)
(277693, 39)
(277829, 1)
(277828, 1)
(1, 1)
(277826, 1

In [36]:
#PUTTING DICTIONARIES FROM ALL 3 INTO ONE
contributor_quality_dict = {}
for i in range(len(pr_all.pull_request_id)):
    if pr_all.pr_cntrb_id[i] in contributor_quality_dict.keys():
        continue
    else:
        contributor_quality_dict[pr_all.pr_cntrb_id[i]] = []
for i in range(len(pc_all.pull_request_commits_id)):
    if pc_all.pc_cntrb_id[i] in contributor_quality_dict.keys():
        continue
    else:
        contributor_quality_dict[pc_all.pc_cntrb_id[i]] = []
for i in range(len(is_all.issue_event_id)):
    if is_all.issue_cntrb_id[i] in contributor_quality_dict.keys():
        continue
    else:
        contributor_quality_dict[is_all.issue_cntrb_id[i]] = []
for key in contributor_quality_dict.keys():
    for k, v in pr_dict_items:
        if key == k:
            contributor_quality_dict[key].append(v)
for key in contributor_quality_dict.keys():
    if len(contributor_quality_dict[key]) == 0:
        contributor_quality_dict[key].append(None)
for key in contributor_quality_dict.keys():
    for k, v in pc_dict_items:
        if key == k:
            contributor_quality_dict[key].append(v)
for key in contributor_quality_dict.keys():
    if len(contributor_quality_dict[key]) == 1:
        contributor_quality_dict[key].append(None)
for key in contributor_quality_dict.keys():
    for k, v in is_dict_items:
        if key == k:
            contributor_quality_dict[key].append(v)
for key in contributor_quality_dict.keys():
    if len(contributor_quality_dict[key]) == 2:
        contributor_quality_dict[key].append(None)
print(contributor_quality_dict)



{280547: [67, 95, 158], 279417: [5, 5, 15], 280551: [37, 38, 91], 280552: [15, 15, 32], 280640: [1, 1, 2], 280642: [1, 1, 2], 280645: [1, 1, 1], 280641: [1, 1, 2], 280631: [1, 1, 1], 280588: [1, 16, 3], 280578: [3, 9, 8], 279422: [5, 7, 6], 280604: [1, 1, 1], 280550: [6, 6, 8], 280603: [1, 1, 1], 280599: [2, 2, 2], 280579: [3, 3, 6], 279418: [4, 4, 5], 280553: [2, 2, 1], 280474: [1, 1, 1], 280521: [27, 28, 46], 279407: [4, 4, 18], 280522: [5, 6, 4], 280523: [329, 492, 518], 280495: [9, 10, 15], 280498: [260, 317, 438], 280507: [221, 220, 330], 279145: [4, 4, 1], 279203: [1, 1, None], 277847: [4, 4, 3], 278656: [6, 6, 2], 279202: [1, 1, None], 280517: [1, 3, 1], 279200: [1, 1, None], 279199: [1, 1, None], 277917: [56, 56, 45], 279192: [3, 4, None], 279781: [13, 13, 23], 279251: [3, 3, 14], 277911: [2, 2, 2], 277705: [11, 14, 11], 277906: [3, 4, 6], 280508: [6, 202, 37], 277915: [3, 3, 4], 277702: [2, 3, 3], 277768: [7, 6, 11], 277914: [1, 1, 1], 277123: [1, 20, 1], 277742: [2, 4, 4], 27

In [37]:
####PRINTING COLLECTED DICTIONARY WITH ALL OUR DATA COLLECTED
print("Contributor ID      Pull Requests     Commits      Issue Events")
for key in contributor_quality_dict.keys():
    pull = contributor_quality_dict[key][0]
    commit = contributor_quality_dict[key][1]
    issue = contributor_quality_dict[key][2]
    print(key, "                ", pull, "            ", commit, "           ", issue)

Contributor ID      Pull Requests     Commits      Issue Events
280547                  67              95             158
279417                  5              5             15
280551                  37              38             91
280552                  15              15             32
280640                  1              1             2
280642                  1              1             2
280645                  1              1             1
280641                  1              1             2
280631                  1              1             1
280588                  1              16             3
280578                  3              9             8
279422                  5              7             6
280604                  1              1             1
280550                  6              6             8
280603                  1              1             1
280599                  2              2             2
280579                  3              3    